In [1]:
import pandas as pd
import numpy as np
import json
import regex
from sqlalchemy import create_engine
import lib
from dotenv import load_dotenv
import os

load_dotenv()
engine = create_engine(os.getenv('POSTGRES'))

/tmp/ipykernel_1395698/4289673733.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
def is_anon(uuid: list) -> list:
    return [True, False] * (len(uuid) // 2)

In [37]:
def transform(bath):
    batch2 = map(lambda x: x['data'].replace("'", '"'), d)
    batch3 = map(json.loads, batch2)
    cols = ['uuid', 'name', 'date', 'region', 'city', 'created_at']
    df = pd.DataFrame(batch3, columns=cols)
    df['anon'] = is_anon(df['uuid'].to_list())
    # df['quality'] = 10
    df['quality_comment'] = ''

    # фильтруем тестовые значения
    mask = df['name'].str.lower().str.contains('test')
    df.loc[mask, 'quality_comment'] = 'test'

    # df.loc[0, 'name'] = df.loc[0, 'name'] + '%'
    # pattern = r'^[A-Za-zА-Яа-я0-9]+$'
    # mask = df['name'].apply(lambda x: bool(re.match(pattern, x)))
    # df.loc[mask, 'quality_comment'] = 'special symbols'

    # в имене больше 3 слов
    mask = df['name'].apply(lambda x: len(x.split()) > 3)
    df.loc[mask, 'quality_comment'] = 'more then 3 words in name'

    # приводим дату к формату даты
    date = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
    mask = df['date'].notna() & date.isna()
    df.loc[mask, 'quality_comment'] = 'bad date'
    df['date'] = date

    # аномизируем часть данных
    mask = df['anon']
    df.loc[mask, 'date'] = df[mask]['date'].dt.to_period('Y').dt.start_time

    names = pd.read_csv('rendom_names.csv')
    names = names['men'].to_list() + names['women'].to_list()
    names_sample = np.random.choice(names, size=mask.sum())
    df.loc[mask, 'name'] = names_sample
    return df
    # удаляем тест
    return df[df['quality_comment'] != 'test']


In [ ]:
lib.create_table()
lib.insert()

In [8]:
def pipeline(size, dist_table_name='topups'):
    for batch in lib.read_by_baches(size=50):
        df = transform(batch)
        df.to_sql(dist_table_name, con=engine, index=False, if_exists='append')


In [10]:
from sqlalchemy import create_engine

In [15]:
engine = create_engine(os.getenv('POSTGRES'))


In [20]:
df2 = pd.read_sql('SELECT * FROM topups', engine)

In [27]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.exc import SQLAlchemyError

In [29]:
metadata = MetaData()
metadata.create_all(bind=engine)

In [55]:
table_to_drop = Table('topups', metadata, autoload_with=engine)
try:
    # Dropping the table
    table_to_drop.drop(bind=engine)
    print(f"Table {table_to_drop.name} dropped successfully.")
except SQLAlchemyError as error:
    print(f"Error while dropping table: {error}")

Table topups dropped successfully.


In [38]:
for batch in lib.read_by_baches(size=50):
    df = transform(batch)
    break

In [44]:
stat = df['quality_comment'].value_counts()

In [53]:
stat.drop('test')

Series([], Name: count, dtype: int64)

In [54]:
stat

quality_comment
test    4
Name: count, dtype: int64

In [49]:
s = 'problemms: ' + str(stat.drop('').to_dict())[1:-1]

"'test': 4"